In [1]:
import numpy as np
import os, sys
sys.path.append(os.path.abspath(os.path.join('../..')))

In [2]:
from utils import solvers

In [4]:
import numpy as np
from scipy.sparse import lil_matrix

# Parameters
N = 81  # grid size
h = 1.0 / (N - 1)  # grid spacing

# Initialize the coefficient matrix A and the source term vector b
A = lil_matrix((N * N, N * N))
b = np.zeros(N * N)

# Loop over each grid point
for i in range(N):
    for j in range(N):
        k = i * N + j
        x = i * h
        y = j * h

        # Apply boundary conditions
        if i == 0:
            A[k, k] = 1
            b[k] = 500 * np.exp(-50 * (1 + y**2))  # Left boundary (x=0)
        elif i == N-1:
            A[k, k] = 1
            b[k] = 100 * (1 - y) + 500 * np.exp(-50 * y**2)  # Right boundary (x=1)
        elif j == 0:
            A[k, k] = 1
            b[k] = 100 * x + 500 * np.exp(-50 * (1 - x)**2)  # Bottom boundary (y=0)
        elif j == N-1:
            A[k, k] = 1
            b[k] = 500 * np.exp(-50 * ((1 - x)**2 + 1))  # Top boundary (y=1)
        else:
            # Interior points
            A[k, k] = -4
            A[k, i * N + (j + 1)] = 1  # right
            A[k, i * N + (j - 1)] = 1  # left
            A[k, (i + 1) * N + j] = 1  # up
            A[k, (i - 1) * N + j] = 1  # down
            b[k] = h**2 * (50000 * np.exp(-50 * ((1 - x)**2 + y**2)) * (100 * ((1 - x)**2 + y**2) - 2))

# Convert A to CSR format for efficient solving
A = A.tocsr()

# Solve the system
phi = np.linalg.solve(A.toarray(), b)

# Reshape phi back into a 2D grid
phi_grid = phi.reshape((N, N))



In [7]:
A.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])